<a href="https://colab.research.google.com/github/divyeeee/CIFAR10-Classification/blob/main/CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
fareselmenshawii_cats_vs_dogs_classification_path = kagglehub.notebook_output_download('fareselmenshawii/cats-vs-dogs-classification')

print('Data source import complete.')


In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import the required libraries
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
# Hyper-parameters
num_epochs = 4
batch_size = 4
learning_rate = 0.001

In [ ]:
# dataset has PILImages images of range [0,1].
# We transform them to Tensors of normalized range [-1,1]
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ]
)

train_dataset = torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)
train_loader  = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,shuffle=True)
test_loader  = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,shuffle=False)

classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

100%|██████████| 170M/170M [00:03<00:00, 45.0MB/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# Implement convolution net
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet,self).__init__()
        self.conv1=nn.Conv2d(3,6,5) #input channel size is 3, 32*32--->28*28
        self.pool = nn.MaxPool2d(2,2) # kernel size is 2 and strides is 2, 14*14
        self.conv2=nn.Conv2d(6,16,5) # 10*10
        # Another pooling operation, 5*5
        self.fc1=nn.Linear(16*5*5, 120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)

    def forward(self,x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x=x.view(-1,16*5*5) # Tensor flattened
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x
        # No softmax, since it has been already included in the CrossEntropy loss handling.

In [ ]:
# Training loop and parameters settings
model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

n_total_steps =len(train_loader)
for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(train_loader):
        # origin shape: [4,3,32,32] = 4,3,1024, 32x32 pixels
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs,labels)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if(i+1) % 2000 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')


Epoch [1/4], Step [2000/12500], Loss: 1.8610
Epoch [1/4], Step [4000/12500], Loss: 1.0515
Epoch [1/4], Step [6000/12500], Loss: 0.8775
Epoch [1/4], Step [8000/12500], Loss: 1.6788
Epoch [1/4], Step [10000/12500], Loss: 1.8057
Epoch [1/4], Step [12000/12500], Loss: 2.1051
Epoch [2/4], Step [2000/12500], Loss: 1.3111
Epoch [2/4], Step [4000/12500], Loss: 1.0797
Epoch [2/4], Step [6000/12500], Loss: 2.2683
Epoch [2/4], Step [8000/12500], Loss: 1.1496
Epoch [2/4], Step [10000/12500], Loss: 0.9495
Epoch [2/4], Step [12000/12500], Loss: 0.9925
Epoch [3/4], Step [2000/12500], Loss: 1.4591
Epoch [3/4], Step [4000/12500], Loss: 0.6501
Epoch [3/4], Step [6000/12500], Loss: 1.2725
Epoch [3/4], Step [8000/12500], Loss: 0.7467
Epoch [3/4], Step [10000/12500], Loss: 1.0616
Epoch [3/4], Step [12000/12500], Loss: 0.8339
Epoch [4/4], Step [2000/12500], Loss: 2.0478
Epoch [4/4], Step [4000/12500], Loss: 1.0457
Epoch [4/4], Step [6000/12500], Loss: 1.4507
Epoch [4/4], Step [8000/12500], Loss: 2.8334
Epoc

In [ ]:
# Model evaluation
with torch.no_grad(): #implies no backward propagation
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value,index)
        _, predicted = torch.max(outputs,1) # torch.max(outputs,1) returns two values, the confidence value or the max score and the index of the confidence score
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if(label == pred):
                n_class_correct[label] +=1
            n_class_samples[label] +=1
    acc = 100.0*n_correct/n_samples
    print(f'Accuracy of the network: {acc}%')

    for i in range(10):
        acc = 100.0*n_class_correct[i]/n_class_samples[i]
        print(f'Accuracy of {classes[i]}:{acc}%')

Accuracy of the network: 55.58%
Accuracy of plane:53.6%
Accuracy of car:70.1%
Accuracy of bird:46.7%
Accuracy of cat:31.4%
Accuracy of deer:50.1%
Accuracy of dog:40.7%
Accuracy of frog:57.1%
Accuracy of horse:63.9%
Accuracy of ship:79.4%
Accuracy of truck:62.8%
